# CTP

综合交易平台（Comprehensive Transaction Platform，CTP）是专门为期货公司开发的一套期货经纪业务管理系统，由交易、风险控制和结算三大系统组成，交易系统主要负责订单处理、行情转发及银期转账业务，结算系统负责交易管理、帐户管理、经纪人管理、资金管理、费率设置、日终结算、信息查询以及报表管理等，风控系统则主要在盘中进行高速的实时试算，以及时揭示并控制风险。系统能够同时连通国内四家期货交易所，支持国内商品期货和股指期货的交易结算业务，并能自动生成、报送保证金监控文件和反洗钱监控文件。

国内程序化交易技术的爆发式发展几乎就是起源于上期技术公司基于 CTP 柜台推出了交易 API，使得用户可以随意开发自己的交易软件直接连接到交易柜台上进行交易，同时 CTP API 的设计模式也成为了许多其他柜台上交易 API 的设计标准，已知的类 CTP 交易 API 参见附录。

CTP 是一套多期货公司共用的交易、结算系统，全部系统部署在上期技术的机房内，由上期技术统一运行维护，不受期货公司的服务器状况的影响。

![CTP](.\images\CTP.PNG)

CTP API 接口包含：
* 交易接口（Trader API）：主要用于获取交易所行情和下达交易指令，如订阅行情，下单，撤单，预埋单，银期转账，信息查询等。交易接口是三个接口中应用最广泛的接口，它的受众主要是终端软件开发商（如快期），对交易终端有特殊需求的个人、机构或自营单位投资者。
* 风控接口（Risk API）：可用于获取场上的实时数据如资金，持仓，保证金占用等，对投资者风险度进行试算，并在必要时采取强平手段以降低风险度，从而为期货公司的风控人员提供交易过程的风险揭示和风险管理服务。上期技术为期货公司风控人员提供的 RcWin 操作平台就是使用风控接口开发而成。由于风控接口所管理的数据是面向所有投资者的，所以风控接口的受众主要是对风控系统有特殊需求的期货公司，而不对个人投资者开放。
* 结算接口（CSV）：是部署在能够访问到综合交易平台物理数据库的电脑上的一系列数据库指令，通过执行相应的指令，操作员可以获取到数据库中的相应数据信息，如投资者资料，成交信息，资金出入信息，持仓信息等。结算接口的主要受众也是期货公司，不针对个人投资者开放。


## CTP 系统架构

![CTP系统架构](.\images\CTP系统架构.PNG)

* 投资者终端：实现了综合交易平台的交易接口和行情接口的交易客户端，提供接收行情、交易、银期转账等功能。
* 交易员终端：实现了综合交易平台 UserAPI 接口，为期货公司交易员提供报单、银期转账、交易数据查询等功能。UserAPI 接口是面向期货公司层面的接口，可一次性控制的数据范围比交易 API 接口大得多，考虑到数据安全和权限划分的问题，暂时不会对期货公司和投资者开放。
* FTD 通讯协议：期货交易数据交换协议。
* 交易前置：交易前置服务一方面通过 TCP 协议与交易终端连接，另一方面通过 FIB 总线与其他后台连接。交易前置主要负责与业务无关的通讯工作，可以分散交易系统的压力，降低交易系统的复杂度，提高安全性。交易前置的功能有三个：链路管理，协议转换和数据路由。
* 行情前置：行情前置一方面通过 FIB 从报盘管理订阅所有行情数据，另一方面通过 TCP 连接把该行情数据转发给订阅了某合约行情数据的交易终端。
* FIB 信息总线：期货交易信息总线，是交易系统的通讯底层构件，为上层应用提供了数据包的封装，请求/应答通讯模式，以及发布/订阅通讯模式。
* 监控系统：旁路了交易系统部分交易数据用以应用数据监控，同时兼顾了交易系统物理部件的容量性能等检测。
* 仲裁服务：指导排队服务的状态切换。
* 排队服务：排队服务负责将交易请求序列化，发布交易序列，作为交易核心（tKernel）处理数据的来源。
* 交易核心：交易核心负责基于投资者的持仓，报单，成交以及出入金情况进行实时的资金和仓位计算的工作，做到事前风控。同时对所有的报单进行校验，驱动交易所报盘接口，及发布实时交易结果到 FIB 总线上。
* 查询核心：内置了与交易核心完全相同的内存数据库结构以及业务规则的实现，基于对投资者实时结算的结果更新内存数据库，通过 FIB 总线为交易客户端提供相关交易数据查询的服务。
* DBMT：与业务数据库进行实时交互，将需要上下场的业务数据通过交易前置送到交易核心进行处理。
* TMDB：向 FIB 总线订阅交易核心的处理结果，将相关的业务数据实时回写到物理数据库中供结算使用。
* 交易初始化：交易初始化主要承担两大职能：1，根据数据库的数据生成交易核心所需的初始化数据；2，向系统发出交易准备指令，使交易系统开始新一轮的交易（trading session）。
* 报盘管理：用于管理交易和行情报盘，使交易核心避免了直接处理报盘接口与交易所前置之间复杂的通讯情况，简化了交易核心的处理逻辑。
* 报盘：实现了交易所的行情和交易 API 接口，通过交易所提供的远程交易席位报单、收取报单、成交回报、以及获取行情等。
* 风控系统：旁路了交易系统排队机发布的交易序列以及交易核心发布的交易结果，对交易数据进行实时监控，同时提供风险试算和强平功能。
* 银期管理：用于管理银期转账接口。
* 清算银行接口（银期接口）：实现了各银行的银期转账接口，为交易系统与银行系统提供数据交互通道。
* 监控中心秘钥接口：用于从保证金监控中心查询期货公司秘钥。
* 监控中心接口管理：用于管理监控中心秘钥接口。
* 业务数据库：为整个系统提供物理数据存储数据来源。
* 管理平台：提供期货公司各种业务操作入口。

下面主要考虑 CTP 中的交易接口，下文主要参考 \cite{VNPY:Tutorial}，\cite{SimNow}。交易接口的主要功能有：

![CTP功能](.\images\CTP功能.PNG)

# 通讯模式

CTP API 使用建立在 TCP 协议之上 FTD 协议（Futures Trading Data Exchange Protocol, 期货交易数据交换协议）与 CTP 后台进行通讯，FTD 协议中的所有通讯都基于某个通讯模式。通讯模式实际上就是通讯双方协同工作的方式。

CTP API 涉及的通讯模式共有三种：
* 对话通讯模式：是指由客户端主动发起的通讯请求，该请求被 CTP 后台接收和处理，并给予响应，如报单、撤单及查询等，这种通讯模式与普通的客户/服务器模式相同。
![对话通讯模式](.\images\对话通讯模式.PNG)
* 私有通讯模式：是指 CTP 后台主动向某个特定的客户端发出的信息，如报单回报、成交回报等。
![私有通讯模式](.\images\私有通讯模式.PNG)
* 广播通讯模式：是指 CTP 后台主动向所有客户端都发出相同的信息，如公告，市场公开信息等。
![广播通讯模式](.\images\广播通讯模式.PNG)

一般，CTP 系统中对话通讯模式下被返回的消息成为**响应**。而私有通讯模式和广播通讯模式下被返回的消息被称为**回报**。

通讯模式和网络的连接不一定存在简单的一对一的关系。也就是说，一个网络连接中可能传送多种不同通讯模式的报文，一种通讯模式的报文也可以在多个不同的连接中传送。

## 数据交换模式

**请求/应答模式**

客户端程序（Client）产生一个请求，发向服务端程序（Server），服务端程序收到后进行处理，并把结果返回给发出请求的客户端程序。

![请求/应答模式](.\images\请求-应答数据交换模式.PNG)

**发布/订阅模式**

发布/订阅模式是一种异步消息传输模式。发布者发布消息到主题，订阅者从主题订阅消息。发布者与订阅者保持相对独立，不需要接触即可保证消息的传送。一个 FIB 应用即可作为发布者，也可作为订阅者。

![发布/订阅模式](.\images\发布-订阅数据交换模式.PNG)


## 数据流

FTD 协议中需要区分的两个重要概念就是通讯模式和数据流。数据流表示的是一个单向或双向的、连续的、没有重复和遗漏的数据报文的序列。通讯模式则是一个数据流进行互动的工作模式。一种实现方式可以为每种通讯模式构造一种数据流，产生了对话流、私有流和广播流。也可以为一个通讯模式建立多种数据流，例如在对话通讯模式下建立两个流：查询流和交易流。广播模式下建立两个流：通知流和行情流。FTD 只规定各个报文在哪个通讯模式下工作，但是不规定数据流的划分。

不同的通讯模式有着不同的数据流管理原则。在对话模式下，一个数据流是一个连接的过程，在这个连接内将保障各个信息的完整性和有序性。但是，当连接断开后，重新连接将开始一个新的数据流，这个数据流和原来的数据流没有直接的关系。如果客户端在提交了一个请求之后，未收到该请求的响应之前断开了连接，则再次连接后，该请求的响应并不会被新的数据流接收。而对于私有模式和广播模式，一个数据流对应一个交易日内的完成某项功能的所有连接。除非强制指定，否则客户端会在重新连接之后，默认的从上次断开连接的地方继续接收下去，而不是从头开始。

每个数据流应该对应一个通讯模式，但是一个通讯模式下可能有多个数据流：
* 对话通讯模式对应对话数据流（DialogRsp）和查询数据流（QueryRsp），对话数据流和查询数据流均为双向数据流，CTP 后台不维护对话数据流和查询数据流的状态，通讯故障时，对话数据流和查询数据流会重置，通讯途中的数据可能会丢失。
* 私有通讯模式对应私有数据流（Private），私有数据流是一个可靠的单向数据流，CTP 后台维护每个登录用户的私有流，在一个交易日内，CTP API 与 CTP 后台断线后恢复连接时，CTP 后台会向使用 restart 或 resume 模式订阅私有数据流的用户重传全部(restart)或断线期间（resume）的私有数据流。
* 广播通讯模式对应公共数据流（Public），公共数据流与私有数据流类似，也是一个可靠的单向数据流。在一个交易日内，CTP API 与 CTP 后台断线恢复连接时，可以请求交易系统发送指定序号之后的公共数据流数据。


## 流文件

综合交易平台接口在初始化时会在本地生成一些流文件。这些流文件用来保存当日客户端程序接收到的公有流，对话流，私有流等报文的数量。流文件主要用来实现 Resume 模式下，重新收取交易所数据的功能，以及在使用综合交易平台风控接口时用来批量查询数据：
* 行情接口实例生成：DialogRsp.con，QueryRsp.con，TradingDay.con
* 交易接口实例生成：DialogRsp.con，Private.con，Public.con，QueryRsp.con，TradingDay.con。

以上流文件的存放路径都是通过接口实例创建函数（CreateFtdcTraderApi\CreateFtdcMdApi）参数指定，如 `CreateFtdcTraderApi(".\\flow\\")` 将会在当前目录的 flow 文件夹存放流文件。客户端无法决定是否生成以上流文件，客户端程序会对流文件进行大量的读写操作，如果客户端不对系统中的句柄数量进行管理的话，很可能出现句柄被用光的情况，因此在开发多 CTP API 实例的客户端时需要注意操作系统的文件句柄限制。另外，流文件中存储了客户端与后台的数据交互进度标识，因此，多账号共用接口实例（或多实例共用流文件）将会造成数据紊乱或缺失。在进行多账户开发时不能将多个账户收取的流文件放在同一个目录下，不然会造成一个账户能收到回报，而其他的账户无法收取回报。

# API 文件

API 文件主要有： 
* .h 文件：C++ 的头文件，包含了 API 的内部结构信息，开发 C++ 程序时需要包含在项目内；
* .dll文件：Windows 下的动态链接库文件，API 的实体，开发 C++ 程序编译和链接时用，使用开发好的程序时也必须放在程序的文件夹内；
* .lib文件：Windows 下的库文件，编译和链接时用，程序开发好后无需放在程序的文件夹内；CTP 的 Lib 文件是标准的 8 个字节对齐的；
* .so文件：Linux 下的动态链接库文件，其他同 .dll 文件。

比如上期 CTP 用于 Windows 系统开发的文件列表为：

![CTP文件](.\images\CTP文件.PNG)

上述文件按功能分为：
* 错误定义文件：error.dtd、error.xml，包含所有可能的错误信息。
* 数据类型定义文件：ThostFtdcUserApiDataType.h，包含了所有用到的数据类型的头文件。
* 数据结构定义文件：ThostFtdcUserApiStruct.h，包含了所有用到的数据结构的头文件。
* 行情：包括行情接口类定义文件：ThostFtdcMdApi.h，行情接口库文件：thostmduserapi.lib，thostmduserapi.dll
* 交易：包括交易接口类定义文件：ThostFtdcTraderApi.h，交易接口库文件：thosttraderapi.lib，thosttraderapi.dll

从用户角度我们只需关注 .h 文件中的内容。对于不同的 API 而言，.h 文件的前缀可能有所区别，如 LTS 是 SecurityFtdc，CTP 是 ThostFtdc。

## 接口类

交易（ThostFtdcTraderApi.h）和行情（ThostFtdcMdApi.h）接口类定义文件都包含 API 和 SPI 两个类的定义：
* Api：比如 CThostFtdcMdApi，CThostFtdcTraderApi，类方法主要为主动函数，包含主动发起请求和订阅的接口函数，客户端使用其向 CTP 后台发送请求，直接调用即可。
* Spi：比如 CThostFtdcMdSpi，CThostFtdcTraderSpi，类方法主要为回调函数，包含有所有的响应和回报函数，用于接收综合交易平台发送或交易所发送综合交易平台转发的信息。开发者需要继承该接口类，并实现其中相应的虚函数。

在运行时，Api，Spi 是不同的线程、Api 可以同时被多个线程调用，这些特性和平台无关。

接口类方法的命名规则：
* 请求：Req——，比如 ReqUserLogin
* 请求的响应：OnRsp——，比如 OnRspUserLogin
* 查询请求：ReqQry——，比如 ReqQryInstrument
* 查询请求的响应：OnRspQry——，比如 OnRspQryInstrument
* 回报：OnRtn——，比如 OnRtnOrder
* 错误回报：OnErrRtn——，比如 OnErrRtnOrderInsert

通用参数：
* nRequestID：客户端发送请求时要为该请求指定一个请求编号。交易接口会在响应或回报中返回与该请求相同的请求编号。当客户端进行频繁操作时，很有可能会造成同一个响应函数被调用多次，这种情况下，能将请求与响应关联起来的纽带就是请求编号。
* bIsLast：当响应函数需要携带的数据包过大时，该数据包会被分割成数个小的数据包并按顺序逐次发送，这种情况下同一个响应函数就是被调用多次，而参数 bIsLast 就是用于描述当前收到的响应数据包是不是所有数据包中的最后一个。例如：如果只有一个数据包，该响应的 bIsLast 值为 true。如果有两个，则第一个响应的 bIsLast 值为 false，第二个为 true。
* pRspInfo：该参数用于描述请求执行过程中是否出现错误。该数据结构中的属性 ErrorID 如果是 0，则说明该请求被交易核心认可通过。否则，该参数描述了交易核心返回的错误信息。error.xml 文件中包含所有可能的错误信息。

注：客户端使用 CTP 的查询函数向后台查询投资者以及基础数据信息，查询条件各字段不设置时多数查询函数会返回对应经纪公司所有记录。如查询合约信息：

    CThostFtdcQryInstrumentField req;
    memset(&req, 0, sizeof(req));
    int iResult = pApi->ReqQryInstrument(&req, ++iRequestID);

这样，在对应的查询响应 OnRspQryInstrument 中将会获得所有有效的合约。

下面分别介绍这 4 个 .h 文件。(以 CTP 的文件为例，详情参见：[CTP-API 接口说明](.\CTP-API-InterfaceDescription.chm))

## UserApiDataType.h

该文件中包含了 API 中用到的常量以及数据类型的定义。

比如以下代码定义了产品名称的数据类型是一个长度为 21 个字符的字符串：

    /////////////////////////////////////////////////////////////////////////
    ///TFtdcProductNameType是一个产品名称类型
    /////////////////////////////////////////////////////////////////////////
    typedef char TThostFtdcProductNameType[21];

再比如以下代码定义了买卖方向常量：

    /////////////////////////////////////////////////////////////////////////
    ///TFtdcDirectionType是一个买卖方向类型
    /////////////////////////////////////////////////////////////////////////
    ///买
    #define THOST_FTDC_D_Buy '0'
    ///卖
    #define THOST_FTDC_D_Sell '1'

## UserApiStruct.h

该文件中包含了 API 中用到的数据结构体的定义。

比如以下代码定义了用户登录请求所发送的数据结构体：

    ///用户登录请求
    struct CThostFtdcReqUserLoginField
    {
        ///交易日
        TThostFtdcDateType	TradingDay;
        ///经纪公司代码
        TThostFtdcBrokerIDType	BrokerID;
        ///用户代码
        TThostFtdcUserIDType	UserID;
        ///密码
        TThostFtdcPasswordType	Password;
        ///用户端产品信息
        TThostFtdcProductInfoType	UserProductInfo;
        ///接口端产品信息
        TThostFtdcProductInfoType	InterfaceProductInfo;
        ///协议信息
        TThostFtdcProtocolInfoType	ProtocolInfo;
        ///Mac地址
        TThostFtdcMacAddressType	MacAddress;
        ///动态密码
        TThostFtdcPasswordType	OneTimePassword;
        ///终端IP地址
        TThostFtdcIPAddressType	ClientIPAddress;
        ///登录备注
        TThostFtdcLoginRemarkType	LoginRemark;
        ///终端IP端口
        TThostFtdcIPPortType	ClientIPPort;
    };

## MdApi.h

该文件中包含了 API 中的行情相关组件的定义。通常开头会有一段和操作系统、编译环境相关的定义。接下来的内容主要包含了两个类 CThostFtdcMdSpi 和 CThostFtdcMdApi

### CThostFtdcMdSpi

MdSpi 类中包含了行情功能相关的回调函数接口，回调函数是柜台端向用户端发送信息后才会被系统自动调用的函数（非用户主动调用）。CThostFtdcMdSpi 的基本结构：

    class CThostFtdcMdSpi{
        public:
            ......

            ///登录请求响应
            virtual void OnRspUserLogin(CThostFtdcRspUserLoginField *pRspUserLogin, CThostFtdcRspInfoField *pRspInfo, int nRequestID, bool bIsLast) {};

            ......

            ///深度行情通知
            virtual void OnRtnDepthMarketData(CThostFtdcDepthMarketDataField *pDepthMarketData) {};
    };

特点：
* 回调函数都是以 On 开头；
* 柜台端向用户端发送的信息经过 API 处理后，传给回调函数的是一个结构体的指针，如 `CThostFtdcRspUserLoginField *pRspUserLogin`，这里的pRspUserLogin 就是一个 C++ 的指针类型，其指向的结构体对象是 CSecurityFtdcRspUserLoginField 结构的，而该结构的定义可以在 ApiStruct.h 中找到；
* 不同的回调函数，传回的参数数量是不同的，OnRspUserLogin 中传入的参数包括两个结构体指针，以及一个整数（代表该响应对应的用户请求号）和一个布尔值（该响应是否是这个请求号的最后一次响应）。


### CThostFtdcMdApi

MdApi 类中包含了行情功能相关的主动函数结构，主动函数指的是由用户负责进行调用的函数，用于向柜台端发送各种请求和指令。CThostFtdcMdApi 的基本结构：

    class MD_API_EXPORT CThostFtdcMdApi{
        public:
            ///创建MdApi
            ///@param pszFlowPath 存贮订阅信息文件的目录，默认为当前目录
            ///@return 创建出的UserApi
            ///modify for udp marketdata
            static CThostFtdcMdApi *CreateFtdcMdApi(const char *pszFlowPath = "");

            ......

            ///注册回调接口
            ///@param pSpi 派生自回调接口类的实例
            virtual void RegisterSpi(CThostFtdcMdSpi *pSpi) = 0;

            ///订阅行情。
            ///@param ppInstrumentID 合约ID  
            ///@param nCount 要订阅/退订行情的合约个数
            ///@remark 
            virtual int SubscribeMarketData(char *ppInstrumentID[], int nCount, char* pExchageID) = 0;

            ......

            ///用户登录请求
            virtual int ReqUserLogin(CThostFtdcReqUserLoginField *pReqUserLoginField, int nRequestID) = 0;

            ......
    };

注：
* MdApi 对象不应该直接创建，而应该通过调用类的静态方法 CreateFtdcMdApi 创建，传入参数为保存 API 的通讯用的 .con 文件的目录（可以选择留空，则 .con 文件会被放在程序所在的文件夹下）。
* 创建 MdSpi 对象后，需要使用 MdApi 对象的 RegisterSpi 方法将该 MdSpi 对象的指针注册到 MdApi 上，也就是告诉 MdApi 从柜台端收到数据后应该通过哪个对象的回调函数推送给用户。（从 API 的这个设计上估计 MdApi 中后包含了和柜台端通讯、接收和发送数据包的功能，而 MdSpi 仅仅是用来实现一个通过回调函数向用户程序推送数据的接口。）
* 绝大部分主动函数在调用时都会用到一个整数类型的参数 nRequestID，该参数在整个 API 的调用中应当保持递增唯一性，从而在收到回调函数推送的数据时，可以知道是由哪次操作引起的。

## TraderApi.h

该文件中包含了 API 中的交易相关组件的定义，其提供了两个接口，分别为 CThostFtdcTraderApi 和 CThostFtdcTraderSpi。这两个接口对 FTD 协议进行了封装，客户端应用程序可以通过 CThostFtdcTraderApi 发出操作请求，通过继承 CThostFtdcTraderSpi 并重载回调函数来处理后台服务的响应。这两个类和 MdApi 中的两个类在结构上非常接近，区别仅仅在于类包含的方法函数上。


### CThostFtdcTraderSpi

CThostFtdcTraderSpi 的基本结构：

    class CThostFtdcTraderSpi{
        public:
            ///当客户端与交易后台建立起通信连接时（还未登录前），该方法被调用。
            virtual void OnFrontConnected(){};

            ...

            ///错误应答
            virtual void OnRspError(CThostFtdcRspInfoField *pRspInfo, int nRequestID, bool bIsLast) {};

            ///登录请求响应
            virtual void OnRspUserLogin(CThostFtdcRspUserLoginField *pRspUserLogin, CThostFtdcRspInfoField *pRspInfo, int nRequestID, bool bIsLast) {};

            ...

            ///报单通知
            virtual void OnRtnOrder(CThostFtdcOrderField *pOrder) {};

            ...

            ///报单录入错误回报
            virtual void OnErrRtnOrderInsert(CThostFtdcInputOrderField *pInputOrder, CThostFtdcRspInfoField *pRspInfo) {};

            ...
    };
Spi（包括 MdSpi 和 TraderSpi）类的回调函数基本上可以分为以下四种：
* 以 On 开头，这种回调函数通常是返回 API 连接相关的信息内容，与业务逻辑无关，返回值（即回调函数的参数）通常为空或是简单的整数类型。
* 以 OnRsp 开头，这种回调函数通常是针对用户的某次特定业务逻辑操作返回信息内容，返回值通常会包括 4 个参数（其中 OnRspError 主要用于一些通用错误信息的返回，因此返回的值中不包含业务逻辑相关结构体指针，只有 3 个返回值）：
    - 业务逻辑相关结构体的指针；
    - 错误信息结构体的指针；
    - 本次操作的请求号整数；
    - 是否是本次操作最后返回信息的布尔值。
* 以 OnRtn 开头，这种回调函数返回的通常是由柜台向用户主动推送的信息内容，如客户报单状态的变化、成交情况的变化、市场行情等等，因此返回值通常只有 1 个参数，为推送信息内容结构体的指针。
* 以 OnErrRtn 开头，这种回调函数通常由于用户进行的某种业务逻辑操作请求（挂单、撤单等等）在交易所端触发了错误，如用户发出撤单指令、但是该订单在交易所端已经成交，返回值通常是 2 个参数，即业务逻辑相关结构体的指针和错误信息的指针。

### CThostFtdcTraderApi

CThostFtdcTraderApi 的基本结构：

    class TRADER_API_EXPORT CThostFtdcTraderApi{
        public:
            ///创建TraderApi
            ///@param pszFlowPath 存贮订阅信息文件的目录，默认为当前目录
            ///@return 创建出的UserApi
            static CThostFtdcTraderApi *CreateFtdcTraderApi(const char *pszFlowPath = "");

            ...

            ///初始化
            ///@remark 初始化运行环境,只有调用后,接口才开始工作
            virtual void Init() = 0;

            ...

            ///用户登录请求
            virtual int ReqUserLogin(CThostFtdcReqUserLoginField *pReqUserLoginField, int nRequestID) = 0;

            ...
    };

Api 类包括的主动函数通常分为以下三种：
* Create...，类的静态方法，用于创建 API 对象，传入参数是用来保存 API 通讯 .con 文件的文件夹路径。
* 以 Req 开头，可以由用户主动调用的业务逻辑请求，传入参数通常包括 2 个：业务请求结构体指针和一个请求号的整数。
* 其他非 Req 开头的函数，包括初始化、订阅数据流等等参数较为简单的功能，传入参数的数量和类型视乎函数功能不一定。


# API 工作流程

行情：
1. 创建回调接口类 MdSpi 对象；调用 MdApi 类的静态方法 CreateFtdcMdApi，创建 MdApi 对象；
2. 调用 MdApi 对象的 RegisterSpi 方法注册 MdSpi 对象的指针；
3. 调用 MdApi 对象的 RegisterFront 方法注册前置机网络地址或者调用 RegisterNameServer 方法注册名字服务器网络地址；
4. 调用 MdApi 对象的 Init 方法初始化到前置机的连接，连接成功后会通过 MdSpi 对象的 OnFrontConnected 回调函数通知用户；连接失败或程序运行过程中该连接断开会通过 MdSpi 对象的 OnFrontDisconnected 回调函数通知用户，其中的参数说明连接失败的原因；
5. 连接成功后，可以调用 MdApi 的 ReqUserLogin 方法登陆，通过 MdSpi 对象的 OnRspUserLogin 通知用户是否登录成功；
6. 登陆成功后，可以调用 MdApi 的 SubscribeMarketData 方法订阅合约，传入参数为想要订阅的合约的代码；客户端发送订阅行情的请求之后，函数 OnRspSubMarketData 会被调用，交易核心认为客户端请求订阅行情的消息不合法时返回错误信息（pRspInfo），订阅消息合法返回的信息则是“CTP：No Error”；订阅成功后，当合约有新的行情时，会通过 MdSpi 的 OnRtnDepthMarketData 回调函数通知用户；
7. 使用 MdApi 对象的 UnSubscribeMarketData 方法退订合约，客户端发送退订合约的请求之后，函数 OnRspUnSubMarketData 会被调用，交易核心认为客户端请求退订行情的消息不合法时返回错误信息（pRspInfo）。
8. 如果交易系统无法识别客户端发送的请求消息，就通过 OnRspError 通知用户，返回错误信息；
9. 如果超过一定时间在客户端和系统之间没有任何消息交换发生，会通过 OnHeartBeatWarning 发送心跳用来说明客户端到系统服务器之间的连接是活跃的。
10. 调用 MdApi 对象的 ReqUserLogout 登出系统，这会先将现有的连接断开。客户端重新登录后系统会再重新建立一个新的连接，而 SessionID 会重置，因此 MaxOrderRef 一般也会重新从 0 计数。

交易：
1. 创建回调接口类 TraderSpi 对象；调用 TraderApi 类的静态方法 CreateFtdcTraderApi，创建 TraderApi 对象；
2. 调用 TraderApi 对象的 RegisterSpi 方法注册 TraderSpi 对象的指针；
3. 订阅公有和私有数据流，调用 TraderApi 对象的 SubscribePublicTopic 和 SubscribePrivateTopic 方法去选择公有和私有数据流的重传方法，默认模式是从上次断开连接处继续收取交易所发布数据（Resume 模式），还可以指定全部重新获取（Restart），或从登陆后获取（Quick）；
4. 调用 TraderApi 对象的 RegisterFront 方法注册前置机网络地址或者调用 RegisterNameServer 方法注册名字服务器网络地址；
5. 调用 TraderApi 对象的 Init 方法初始化到前置机的连接，连接成功后会通过 TraderSpi 对象的 OnFrontConnected 回调函数通知用户；连接失败或程序运行过程中该连接断开会通过 TraderSpi 对象的 OnFrontDisconnected 回调函数通知用户，其中的参数说明连接失败的原因；
6. 身份认证（对于期货柜台比如 CTP、恒生 UFT 期货等需要，证券柜台 LTS 无此要求），调用 TraderApi 对象的 ReqAuthenticate 方法发起客户端认证请求，客户端认证结果由 OnRspAuthenticate 返回。身份认证功能是否启用在期货公司的业务人员使用的结算平台上是可以进行配置的。期货公司可以选择关闭身份认证功能，则客户端可不必进行身份认证。否则期货公司需要在结算平台上维护该客户端程序的认证码（AuthCode）；
7. 客户端在本次会话主动发起认证成功或者 CTP 后台开启强制客户端认证后，调用 TraderApi 对象的 ReqUserLogin 方法登陆，登陆成功后会通过 TraderSpi 对象的 OnRspUserLogin 回调函数通知用户；
8. 在每日第一次登陆成功后需要先查询前一日的结算单（ReqQrySettlementInfo），等待结算单查询结果返回（OnRspQrySettlementInfo）后，确认结算单（ReqSettlementInfoConfirm）成功（OnRspSettlementInfoConfirm）后，才可以进行交易；在一天中，如果投资者中已经确认了结算结果，以后就不再必须确认结算结果，就可以直接发送交易指令了。
9. 在正式开始交易之前，客户端可能还需要查询如下基础数据，包括：
    * ReqQryInstrument：查询所有可以交易的合约信息，包括代码、中文名、涨跌停、最小价位变动、合约乘数等，一般是在这里获得合约信息列表后，再去 MdApi 中订阅合约；
    * ReqQryTradingAccount：查询资金
    * ReqQryInvestorPosition：查询持仓
    * ReqQryOrder：查询报单，支持分时段查询。
    * ReqQryTrade：查询成交，支持分时段查询。
10. 当用户的报单、成交状态发生变化时，TraderApi 会自动通过 OnRtnOrder、OnRtnTrade 通知用户，无需额外订阅；用户的某次请求发生错误时，会通过 OnRspError 通知用户。

![CTP 初始化过程](.\images\CTP初始化过程.PNG)

# 用户登录

登入请求：`int ReqUserLogin(CThostFtdcReqUserLoginField *pReqUserLoginField, int nRequestID)`

用户登入时需要输入：
* 经纪公司代码：`TThostFtdcBrokerIDType BrokerID;`
* 用户代码，即投资者代码：`TThostFtdcUserIDType UserID;`
* 密码：`TThostFtdcPasswordType Password;`
* 用户端产品信息，就是终端程序的名称：`TThostFtdcProductInfoType UserProductInfo;`

登入成功后，会收到登入响应：`void OnRspUserLogin(CThostFtdcRspUserLoginField *pRspUserLogin, CThostFtdcRspInfoField *pRspInfo, int nRequestID, bool bIsLast)`，第一个参数 pRspUserLogin 中包含了服务器端返回的一些基础数据：
* 当前会话的参数。用户可以用这些参数定义自己的交易序列号 FrontID+SessionID+OrderRef
    - 前置编号：`TThostFtdcFrontIDType FrontID;`
    - 会话编号：`TThostFtdcSessionIDType SessionID;`
    - 最大报单引用：`TThostFtdcOrderRefType MaxOrderRef;`
* 当前各个交易所的时间。终端程序依此可以估计未来各个交易所的时间。
    - 上期所时间：`TThostFtdcTimeType SHFETime;`
    - 大商所时间：`TThostFtdcTimeType DCETime;`
    - 郑商所时间：`TThostFtdcTimeType CZCETime;`
    - 中金所时间：`TThostFtdcTimeType FFEXTime;`
    - 上海能源中心时间：`TThostFtdcTimeType INETime;`

函数 ReqUserLogout 用来退出登录，用法和上述的请求登录操作一致。目前，通过 ReqUserLogout 登出系统的话，会先将现有的连接断开。客户端重新登录后系统会再重新建立一个新的连接，而 SessionID 会重置，因此 MaxOrderRef 一般也会重新从 0 计数。

# 行情订阅

订阅行情：`int SubscribeMarketData(char *ppInstrumentID[], int nCount)`，第一个参数是一个包含所有要订阅的合约的数组，第二个参数是该数组的长度。

客户端发送订阅行情的请求之后，函数 OnRspSubMarketData 和函数 OnRtnDepthMarketData 将会被调用。

如果客户端订阅行情的请求是不合法的，OnRspSubMarketData 返回服务器端给出的错误信息（pRspInfo）。即使客户端发送的订阅请求是合法的，OnRspSubMarketData 也会被调用，而返回的信息则是“CTP：No Error”。

行情订阅请求是合法的，服务端通过调用 `virtual void OnRtnDepthMarketData(CThostFtdcDepthMarketDataField *pDepthMarketData)` 直接返回某合约的市场行情。频率是每秒两次（500 毫秒一次）。其中，CThostFtdcDepthMarketDataField 的字段如下：

    ///深度行情
    struct CThostFtdcDepthMarketDataField
    {
        ///交易日
        TThostFtdcDateType TradingDay;
        ///合约代码
        TThostFtdcInstrumentIDType InstrumentID;
        ///交易所代码
        TThostFtdcExchangeIDType ExchangeID;
        ///合约在交易所的代码
        TThostFtdcExchangeInstIDType ExchangeInstID;
        ///最新价
        TThostFtdcPriceType	LastPrice;
        ///上次结算价
        TThostFtdcPriceType	PreSettlementPrice;
        ///昨收盘
        TThostFtdcPriceType	PreClosePrice;
        ///昨持仓量
        TThostFtdcLargeVolumeType PreOpenInterest;
        ///今开盘
        TThostFtdcPriceType	OpenPrice;
        ///最高价
        TThostFtdcPriceType	HighestPrice;
        ///最低价
        TThostFtdcPriceType	LowestPrice;
        ///数量
        TThostFtdcVolumeType Volume;
        ///成交金额
        TThostFtdcMoneyType	Turnover;
        ///持仓量
        TThostFtdcLargeVolumeType OpenInterest;
        ///今收盘
        TThostFtdcPriceType	ClosePrice;
        ///本次结算价
        TThostFtdcPriceType	SettlementPrice;
        ///涨停板价
        TThostFtdcPriceType	UpperLimitPrice;
        ///跌停板价
        TThostFtdcPriceType	LowerLimitPrice;
        ///昨虚实度
        TThostFtdcRatioType	PreDelta;
        ///今虚实度
        TThostFtdcRatioType	CurrDelta;
        ///最后修改时间
        TThostFtdcTimeType UpdateTime;
        ///最后修改毫秒
        TThostFtdcMillisecType UpdateMillisec;
        ///申买价一
        TThostFtdcPriceType	BidPrice1;
        ///申买量一
        TThostFtdcVolumeType BidVolume1;
        ///申卖价一
        TThostFtdcPriceType	AskPrice1;
        ///申卖量一
        TThostFtdcVolumeType AskVolume1;
        ///申买价二
        TThostFtdcPriceType	BidPrice2;
        ///申买量二
        TThostFtdcVolumeType BidVolume2;
        ///申卖价二
        TThostFtdcPriceType	AskPrice2;
        ///申卖量二
        TThostFtdcVolumeType AskVolume2;
        ///申买价三
        TThostFtdcPriceType	BidPrice3;
        ///申买量三
        TThostFtdcVolumeType BidVolume3;
        ///申卖价三
        TThostFtdcPriceType	AskPrice3;
        ///申卖量三
        TThostFtdcVolumeType AskVolume3;
        ///申买价四
        TThostFtdcPriceType	BidPrice4;
        ///申买量四
        TThostFtdcVolumeType BidVolume4;
        ///申卖价四
        TThostFtdcPriceType	AskPrice4;
        ///申卖量四
        TThostFtdcVolumeType AskVolume4;
        ///申买价五
        TThostFtdcPriceType	BidPrice5;
        ///申买量五
        TThostFtdcVolumeType BidVolume5;
        ///申卖价五
        TThostFtdcPriceType	AskPrice5;
        ///申卖量五
        TThostFtdcVolumeType AskVolume5;
        ///当日均价
        TThostFtdcPriceType	AveragePrice;
        ///业务日期
        TThostFtdcDateType	ActionDay;
    };

其中：
* TradingDay, UpdateTime, UpdateMillisec 分别表示日期（`char[9]`, yyyymmdd）, 时间(`char[9]`，HH:MM:SS)，毫秒（int）。
* 真实环境中返回的成交金额 Turnover 及当日均价 AveragePrice 两个字段不正确，交易客户端用到该数据时需要自行调整，调整规则如下：
    - 郑商所：当日均价正确，成交金额需乘以合约乘数；
    - 大商所：当日均价需除以合约乘数，成交金额正确；
    - 上期所：当日均价需除以合约乘数，成交金额正确。
* 对于夜盘结束等停止交易的时段，交易所可能会把最后一条行情反复推送。

函数 UnSubscribeMarketData 用来退订行情数据，用法和上述的请求订阅行情操作一致。

# 交易指令

## 报单流程

![报单流程](.\images\报单流程.PNG)

1. 交易终端通过交易接口向交易前置提交报单申请。
2. 主排队机从前置机订阅交易申请报文。
3. 主排队机将发布的交易序列报文发送给从排队机要求确认。
4. 从排队机收到待确认报文后将相关报文写入流水文件，并立即返回报文确认信息。
5. 交易核心从主排队机订阅交易序列报文。
6. 交易核心对收到的交易序列报文做合法性检查，检查出错误的交易申请报文后就会返回给交易前置一个包含错误信息的报单响应报文，交易前置立即将该报文信息转发给交易终端。如果检出为合法的交易申请报文，交易核心也会返回一个报单响应报文到交易前置，但是该报文不会被交易前置返回给交易终端。
7. 两种情况：
    - 交易前置从交易核心订阅到错误的报单响应报文，以对话模式将该报文转发给交易终端。
    - 交易核心返回给交易前置的响应报文是正确的，交易前置立即以私有模式返回对应报单的报单回报到交易终端（图中前置机到交易终端 8）。
8. 两个过程：
    - 交易前置在订阅到交易核心的报单回报后，以私有模式将该报单回报发送到交易终端。
    - 交易核心向交易前置发送了第一个报单回报后，立即产生向交易所申请该报单插入的申请报文，该报文被报盘管理订阅。
9. 报盘管理订阅到交易所报盘插入申请报文后，将该报文转发到对应报盘接口。
10. 报盘收到报盘管理的报单申请报文后，通过交易所提供的交易接口将该笔报单发送到交易所。
11. 报盘通过交易接口从交易所前置接收报单回报以及成交回报或出错的报单响应报文。
12. 报盘将从交易所接收到的报单回报及成交回报或报单响应汇总到报盘管理。
13. 主排队机从报盘管理订阅从交易所返回的报单信息。
14. 主排队机将报文信息序列化后发送给从排队机进行确认。
15. 从排队机收到待确认报文后将该报文写入流水，并立即返回报文确认信息。
16. 交易核心从主排队机订阅所有的报单以及成交回报信息。
17. 交易前置从交易核心订阅所有交易核心发布的交易结果数据。
18. 交易前置将订阅到的交易结果数据分发到各交易终端。

![报撤单过程](.\images\报撤单过程.PNG)

## 报单指令

报单指令是：`int ReqOrderInsert(CThostFtdcInputOrderField *pInputOrder, int nRequestID)`。

CThostFtdcInputOrderField 为输入报单结构体，报单指令中如下字段需要如下设置：
* 买卖方向：Direction，可选值：
    - 买：THOST_FTDC_D_Buy
    - 卖：THOST_FTDC_D_Sell
* 成交量类型：VolumeCondition，可选值：
    - THOST_FTDC_VC_AV：任何数量
    - THOST_FTDC_VC_MV：最小数量
    - THOST_FTDC_VC_CV：全部数量
* 最小成交量：MinVolume，成交量类型是最小数量时需设置该值，否则设为 1。
* 报单价格类型：OrderPriceType，可选值：
    - 任意价：THOST_FTDC_OPT_AnyPrice
    - 限价：THOST_FTDC_OPT_LimitPrice
    - 最优价：THOST_FTDC_OPT_BestPrice
    - 最新价：THOST_FTDC_OPT_LastPrice
    - 最新价上浮 1 个 Tick：THOST_FTDC_OPT_LastPricePlusOneTicks
    - 最新价上浮 2 个 Tick：THOST_FTDC_OPT_LastPricePlusTwoTicks
    - 最新价上浮 3 个 Tick：THOST_FTDC_OPT_LastPricePlusThreeTicks
    - 卖一价：THOST_FTDC_OPT_AskPrice1
    - 卖一价上浮 1 个 Tick：THOST_FTDC_OPT_AskPrice1PlusOneTicks
    - 卖一价上浮 2 个 Tick：THOST_FTDC_OPT_AskPrice1PlusTwoTicks
    - 卖一价上浮 3 个 Tick：THOST_FTDC_OPT_AskPrice1PlusThreeTicks
    - 买一价：THOST_FTDC_OPT_BidPrice1
    - 买一价上浮 1 个 Tick：THOST_FTDC_OPT_BidPrice1PlusOneTicks
    - 买一价上浮 2 个 Tick：THOST_FTDC_OPT_BidPrice1PlusTwoTicks
    - 买一价上浮 3 个 Tick：THOST_FTDC_OPT_BidPrice1PlusThreeTicks
    - 五档价：THOST_FTDC_OPT_FiveLevelPrice
* 价格：LimitPrice，报单价格类型是限价时需设置该值，否则设为 0。
* 开平标志：CombOffsetFlag，该字段是一个长度为 5 的字符数组，可以同时用来描述单腿合约和组合合约的报单属性。单腿合约只需要为数组的第 1 个元素赋值，组合合约需要为数组的第 1 和 2 个元素赋值。对于平仓，上期所、能源交易所区分昨仓和今仓，平昨仓设置为 THOST_FTDC_OF_Close 或 THOST_FTDC_OF_CloseYesterday，平今仓设置为 THOST_FTDC_OF_CloseToday，其他交易所不区分昨仓和今仓，THOST_FTDC_OF_Close 或 THOST_FTDC_OF_CloseToday 或 THOST_FTDC_OF_CloseYesterday 均可。可选值：
    - 开仓：THOST_FTDC_OF_Open
    - 平仓：THOST_FTDC_OF_Close
    - 平今：THOST_FTDC_OF_CloseToday
    - 平昨：THOST_FTDC_OF_CloseYesterday
    - 强减：THOST_FTDC_OF_ForceOff
    - 强平：THOST_FTDC_OF_ForceClose
    - 本地强平：THOST_FTDC_OF_LocalForceClose
* 有效期：TimeCondition，可选值：
    - 立即完成，否则撤销：THOST_FTDC_TC_IOC
    - 本节有效：THOST_FTDC_TC_GFS
    - 当日有效：THOST_FTDC_TC_GFD
    - 指定日期前有效：THOST_FTDC_TC_GTD
    - 撤销前有效：THOST_FTDC_TC_GTC
    - 集合竞价有效：THOST_FTDC_TC_GFA
* 自动挂起标志：IsAutoSuspend = 0，否
* 用户强平标志：UserForceClose = 0，否
* 强平原因：ForceCloseReason = THOST_FTDC_FCC_NotForceClose，非强平
* 投机套保标志：CombHedgeFlag[0]= THOST_FTDC_BHF_Speculation，投机

**限价单**
* 报单价格类型：OrderPriceType = THOST_FTDC_OPT_LimitPrice，限价
* 价格：LimitPrice = ...，用户设定
* 成交量类型：VolumeCondition = THOST_FTDC_VC_AV，任何数量
* 有效期：TimeCondition = THOST_FTDC_TC_GFD，当日有效


**市价单**
* 报单价格类型：OrderPriceType = THOST_FTDC_OPT_AnyPrice，任意价
* 价格：LimitPrice = 0
* 成交量类型：VolumeCondition = THOST_FTDC_VC_AV，任何数量
* 有效期：TimeCondition = THOST_FTDC_TC_IOC，立即完成，否则撤销

**条件单**

条件单是一个带有触发条件的指令。该触发条件可以以市场上的最新行情为基准，也可以以时间为基准。比如希望在市场价低于某个价位时买入，就可以使用条件单。这样当行情波动到满足该条件时，该报单就会被自动触发报出，而不需要时刻盯着电脑屏幕去监视市场行情。有效的使用条件单，可以做出限价止损指令（stop-and-limit order）和触及市价指令（market-if-touched order）：
* 报单价格类型：OrderPriceType = THOST_FTDC_OPT_LimitPrice，限价
* 价格：LimitPrice = ...，用户设定
* 有效期：TimeCondition = THOST_FTDC_TC_GFD，当日有效
* 触发条件：ContingentCondition，可选值有：
    - THOST_FTDC_CC_Immediately：立即
    - THOST_FTDC_CC_Touch：止损
    - THOST_FTDC_CC_TouchProfit：止盈
    - THOST_FTDC_CC_ParkedOrder：预埋单
    - THOST_FTDC_CC_LastPriceGreaterThanStopPrice：最新价大于条件价
    - THOST_FTDC_CC_LastPriceGreaterEqualStopPrice：最新价大于等于条件价
    - THOST_FTDC_CC_LastPriceLesserThanStopPrice：最新价小于条件价
    - THOST_FTDC_CC_LastPriceLesserEqualStopPrice：最新价小于等于条件价
    - THOST_FTDC_CC_AskPriceGreaterThanStopPrice：卖一价大于条件价
    - THOST_FTDC_CC_AskPriceGreaterEqualStopPrice：卖一价大于等于条件价
    - THOST_FTDC_CC_AskPriceLesserThanStopPrice：卖一价小于条件价
    - THOST_FTDC_CC_AskPriceLesserEqualStopPrice：卖一价小于等于条件价
    - THOST_FTDC_CC_BidPriceGreaterThanStopPrice：买一价大于条件价
    - THOST_FTDC_CC_BidPriceGreaterEqualStopPrice：买一价大于等于条件价
    - THOST_FTDC_CC_BidPriceLesserThanStopPrice：买一价小于条件价
    - THOST_FTDC_CC_BidPriceLesserEqualStopPrice：买一价小于等于条件价
* 止损价：StopPrice，用于触发条件的价格

**FOK（Fill or Kill）**

FOK 是一种特殊的报单类型，该报单被交易所接收后，交易所会扫描市场行情，如果在当时的市场行情下该报单可以立即全部成交，则该报单会参与撮合成交，否则立即全部撤销：
* 报单价格类型：OrderPriceType = THOST_FTDC_OPT_LimitPrice，限价
* 价格：LimitPrice = ...，用户设定
* 有效期：TimeCondition = THOST_FTDC_TC_IOC，立即完成，否则撤销
* 成交量类型：VolumeCondition = THOST_FTDC_VC_CV，全部数量

**FAK（Fill and Kill）**

FAK 是一种特殊的报单类型，该报单被交易所接收后，交易所会扫描市场行情，在当时的行情下能立即成交多少手即参与撮合成交多少手，剩余的则立即全部撤销：
* 报单价格类型：OrderPriceType = THOST_FTDC_OPT_LimitPrice，限价
* 价格：LimitPrice = ...，用户设定
* 有效期：TimeCondition = THOST_FTDC_TC_IOC，立即完成，否则撤销
* 成交量类型：VolumeCondition = THOST_FTDC_VC_AV（任何数量）或者 THOST_FTDC_VC_MV（最小数量），若为后者，需要指定最小成交量，立即能成交的手数如果小于该数量，则不会参与撮合成交，全部立即撤销。


### 报单序列号

在综合交易平台和交易所中，每笔报单都有以下 3 组唯一序列号，保证其与其他报单是不重复的：
* FrontID、SessionID、OrderRef：用户使用这组交易序列号可以按照自己的方式来唯一标示发出的任何一笔报单。
    - 用户登入成功后，会收到前置机编号 FrontID, 会话编号 SessionID 和最大报单引用 MaxOrderRef。
    - 用户在报单时设定报单引用 OrderRef。OrderRef 可以从 MaxOrderRef 开始递增。如果用户没有设定 OrderRef，在报单响应中，Thost 会为用户设置一个 OrderRef，使得每个报单的这组序列号保持唯一。
    - 在没有得到报单响应前，可以使用这组交易序列号进行撤单操作。
* ExchangeID、TraderID、OrderLocalID：交易席位在向交易所报单时，产生这组交易序列号，标示每一笔发往交易所的报单。交易核心将报单提交到报盘管理之后由交易核心生成 OrderLocalID 并返回给客户端的。ExchangeID 为合约所在交易所的代码，TraderID 由交易核心选定返回。客户端也可以通过这组序列号进行撤单操作。与第一组序列号不同的是：该序列号是由综合交易平台的交易核心维护。
* ExchangeID、OrderSysID：交易所接受了投资者报单，会为该报单生成报单在交易所的编号 OrderSysID，ExchangeID 是固定的，标示每一笔收到的报单。再经由综合交易平台转发给客户端。客户端也可以通过这组序列号进行撤单操作。这组序列号由交易所维护。

另外，Thost 收到用户报单后，为每个经纪公司的报单生成一组交易序列号：BrokerID、BrokerOrderSeq。


### 报单响应和回报

OnRspOrderInsert：Thost 收到报单指令，如果没有通过参数校验，拒绝接受报单指令。用户就会收到 OnRspOrderInsert 消息，其中包含了错误编码和错误消息。如果 Thost 接受了报单指令，用户不会收到 OnRspOrderInsert，而会收到 OnRtnOrder，用来更新委托状态。

OnErrRtnOrderInsert：报盘将通过交易核心检查的报单发送到交易所前置，交易所会再次校验该报单。如果交易所认为该报单不合法，交易所会将该报单撤销，将错误信息返回给报盘，并返回更新后的该报单的状态。当客户端接收到该错误信息后，就会调用 OnErrRtnOrderInsert 函数，而更新后的报单状态会通过调用函数 OnRtnOrder 发送到客户端。如果交易所认为该报单合法，则只通过 OnRtnOrder 返回该报单状态（“尚未触发”）。

OnRtnOrder：交易系统返回的报单状态，每次报单状态发生变化时被调用。一次报单过程中会被调用数次，比如交易系统将报单向交易所提交时，交易所撤销或接受该报单时，该报单成交时等，如果报单是分笔成交，则每次成交都会有一次 OnRtnOrder 返回。报单回报描述了报单的当前状态，其中包括：
- 原始的报单指令
- 报单序列号：FrontID、SessionID、OrderRef，BrokerID、BrokerOrderSeq，ExchangeID、TraderID、LocalOrderID，ExchangeID、OrderSysID；
- 报单委托状态
- 报单数量状态：其中，VolumeTotalOriginal、VolumeTraded、VolumeTotal 分别对应该报单的原始报单数量、已成交数量和剩余数量。

OnRtnErrorConditionalOrder：条件单触发时，交易核心会对该报单的合法性进行校验，如果校验失败，通过该返回校验失败的错误信息。

Thost 支持同一个投资者帐号创建多个 TradeApi 对象同时登入系统进行交易。为区分各自的委托回报和成交回报，可以采用如下方式：
* 登入成功后，记下当前会话的 FrontID、SessionID.
* 报单时，自己设定 OrderRef。
* 收到报单回报时，使用 FrontID、SessionID 过滤出自己的报单回报。同时记下关联的 ExchangeID、OrderSysID。
* 收到成交回报时，使用以上得到的 ExchangeID、OrderSysID 过滤出自己的成交回报。
注意：上述方法只在当前会话中的报单有效。


### 成交回报

如果该报单由交易所进行了撮合成交，交易所再次通过 OnRtnOrder 返回该报单的状态（“已成交”），并通过 OnRtnTrade 返回该笔成交回报。每笔成交都会调用一次成交回报。成交回报中只包含合约、成交数量、价格等与该笔成交相关的信息，并不包含该笔成交之后投资者的持仓、资金等信息。

成交回报描述了报单的成交事件，包括分笔成交。其中包括：
* 交易序列号：BrokerID、BrokerOrderSeq，ExchangeID、TraderID、LocalOrderID，ExchangeID、OrderSysID；

已知一笔委托的 FrontID、SessionID、OrderRef，要在成交回报中找到相关的成交记录。可以在报单回报中，从 FrontID、SessionID、OrderRef 印射到相关的 ExchangeID、OrderSysID。然后在成交回报中，用 ExchangeID、OrderSysID 找出这笔委托的相关成交记录。

报单成交之后，一个报单回报（OnRtnOrder）和一个成交回报（OnRtnTrade）会被发送到客户端，建议客户端将成交回报作为报单成交的标志，因为 CTP 的交易核心在收到 OnRtnTrade 之后才会更新该报单的状态。如果客户端通过报单回报来判断报单成交与否并立即平仓，有极小的概率会出现在平仓指令到达 CTP 交易核心时该报单的状态仍未更新，就会导致无法平仓。

## 撤单指令

撤单指令是：`ReqOrderAction(CThostFtdcInputOrderActionField *pInputOrderAction, int nRequestID)`，输入参数CThostFtdcInputOrderActionField：
* 撤单操作引用：OrderActionRef = ...，与 OrderRef 相似，由用户自己设定，保持递增。如果用户不设定的话，由 Thost 来设定。
* 操作标志类型：ActionFlag = THOST_FTDC_AF_Delete，撤单，由于国内的交易所目前只支持撤单，不支持改单操作，因此函数 ReqOrderAction 只支持撤单操作，字段 ActionFlag 的赋值目前只能是 THOST_FTDC_AF_Delete。
* 报单序列号：FrontID、SessionID、OrderRef、ExchangeID、OrderSysID，这 5 个序列号在撤单时都要使用，如果报单还停留在 Thost，Thost 可以用 FrontID、SessionID、OrderRef 来定位；如果报单停留在交易所，Thost 可以用 ExchangeID、OrderSysID 来定位，然后向交易所转发撤单指令。
* 其他参数：BrokerID、UserID、InvestorID、InstrumentID。

### 撤单响应和回报

OnRspOrderAction：交易核心收到撤单指令，如果没有通过参数校验，拒绝接受撤单指令。用户就会收到 OnRspOrderAction 消息，其中包含了错误编码和错误消息。

OnRtnOrder：交易核心确认了撤单指令的合法性后，将该撤单指令提交给交易所，用户不会收到 OnRspOrderAction，而会收到 OnRtnOrder，用来更新报单状态。

OnErrRtnOrderAction：交易所会再次验证撤单指令的合法性，如果交易所认为该指令不合法，交易核心通过此函数转发交易所给出的错误。如果交易所认为该指令合法，同样会返回对应报单的新状态（OnRtnOrder）。

## 预埋单

预埋单是一种能且仅能在非交易时段（集合竞价前或交易节之间的休息时间）报入并在新的交易时段开始时被触发并执行一定指令的一种指令。它包含预埋报单和预埋撤单：
* 预埋报单（Parked Order Insert）被触发时，一个新的报单被报入交易所。
* 预埋撤单（Parked Order Action）被触发时，一个撤单指令被报入交易所，请求撤销某笔已经存在的报单。

预埋报单或撤单在被触发后即转化为一个普通的报单或撤单指令，之后的处理过程与报单或撤单过程完全一样。目前，预埋单被触发后报入交易所的报单的报单引用是由综合交易平台生成，暂时不受客户端控制。

预埋报单使用的函数是 ReqParkedOrderInsert。预埋报单的用法与普通报单类似。对应的响应函数为 OnRspParkedOrderInsert，该函数用于返回交易核心给出的响应。删除预埋报单使用的函数是 ReqRemoveParkedOrder，该函数用于删除已经报入但未触发的某笔预埋报单。

预埋撤单使用的函数是 ReqParkedOrderAction。对应的响应函数为 OnRspParkedOrderAction。删除预埋撤单使用的函数是 ReqRemoveParkedOrderAction，该函数用于删除已经报入但未触发的某笔预埋撤单。

函数 ReqQryParkedOrder 和函数 ReqQryParkedOrderAction 分别用于查询预埋报单和预埋撤单情况。对应的响应函数分别是 OnRspQryParkedOrder 和 OnRspQryParkedOrderAction。

## 其他指令

### 资金状况

使用 ReqQryTradingAccount。查询投资者最新的资金状况，比如保证金，手续费，持仓盈利，可用资金等。

### 合约保证金率

使用的函数是 ReqQryInstrumentMarginRate。该函数只能查询单腿合约的保证金率。组合合约的保证金率可以通过查询两条单腿合约的保证金率，然后通过交易所的规则去计算。

### 行权

行权指令是：`int ReqExecOrderInsert(CThostFtdcInputExecOrderField *pInputExecOrder, int nRequestID)`，CThostFtdcInputExecOrderField：


如果交易核心认为该行权指令不合法，函数 OnRspExecOrderInsert 会被调用。如果交易核心认为该行权指令合法，函数 OnRtnExecOrder 会被调用。

行权之后，如果收到返回信息“未执行”，则说明该行权指令已经被交易所接收。由于交易所都是在盘后结算时进行行权配对，所以盘中行权返回“未执行”的信息。

交易所对期权行权的处理：
* 中金所：实值期权自动行权，虚值期权自动放弃。虚值期权不允许强制执行。期权行权之后立即平仓（因为期权行权日与对应标的期货交割日为同一天）。
* 上期所：实值期权自动行权，虚值期权自动放弃。投资者可以主动申请执行虚值期权。期权行权之后可以选择保留或不保留期货持仓。
* 大商所：期货公司在会服（大商所官方网站）设置了自动行权时会自动行权，否则需要投资者主动申请行权或放弃。
* 郑商所：实值期权自动行权，虚值期权自动放弃。投资者可以主动申请执行或放弃。

# 测试系统

CTP 提供证券模拟交易系统供投资者开发、测试试用:
* 交易前置: ctp24-front1.financial-trading-platform.com:41205 或 ctp24-front2.financial-trading-platform.com:41205
* 行情前置: ctp24-front3.financial-trading-platform.com:41213 或 ctp24-front4.financial-trading-platform.com:41213
* 经纪公司代码：2011
* 账号：1000021-1000030 共 10 个公用帐号，密码均为 1。

CTP 提供期货模拟交易系统供投资者开发、测试试用:
* 交易前置: asp-sim2-front1.financial-trading-platform.com:26205
* 行情前置: asp-sim2-md1.financial-trading-platform.com:26213
* 经纪公司代码：2030
* 账号：1000021-1000030 共 10 个公用帐号，密码均为 1。

国内证券、期货市场正常交易时间均可交易，每个交易日晚 17:30 到凌晨 05:00 也可进行交易，节假日正常情况下都可进行交易。模拟环境只有上期所的交易所系统，其他交易所的合约也是在上期所系统模拟。

# 模拟交易系统

BrokerID 统一为：9999

第一套：
* 标准CTP：
    - 第一组：Trade Front：180.168.146.187:10000，Market Front：180.168.146.187:10010；【电信】
    - 第二组：Trade Front：180.168.146.187:10001，Market Front：180.168.146.187:10011；【电信】
    - 第三组：Trade Front：218.202.237.33 :10002，Market Front：218.202.237.33 :10012；【移动】
    - 交易阶段(服务时间)：与实际生产环境保持一致
* CTP FIX 网关：
    - 第一组：Trade Front:180.168.146.187:10056，Market Front：180.168.146.187:13050；【电信】
    - 交易阶段(服务时间)：与实际生产环境保持一致。

第二套：
* 交易前置：180.168.146.187:10030
* 行情前置：180.168.146.187:10031；【7x24】
* 交易阶段(服务时间)：交易日：16:00～次日 09:00；非交易日：16:00～次日 15:00。

第二套环境仅服务于 CTP API 开发爱好者，仅为用户提供 CTP API 测试需求，不提供结算等其它服务。新注册用户，需要等到第二个交易日才能使用第二套环境。账户、钱、仓跟第一套环境上一个交易日保持一致。用户通过 SimNow 的账户（上一个交易日之前注册的账户都有效）接入环境，建议通过商业终端进行模拟交易的用户使用第一套环境。


**成交规则**
* 期货交易按照交易所公布的买一卖一价对价成交；
* 买入时：如果委托价大于等于卖一价，则成交，成交价为委托价、卖一价、最新价三价取中，如果委托价小于卖一价，不能成交，等待更优的行情才能成交；
* 卖出时：如果委托价小于等于买一价，则成交，成交价为委托价、买一价、最新价三价取中，如果委托价大于买一价，不能成交，等待更优的行情才能成交。

**清算规则**
每日结算的结算价取自真实市场的结算价，在最后交易日时现金交割（以合约当天结算价了结剩余仓位）。

# API 使用陷阱

## 断线重连

客户端与服务端断开连接时，函数 OnFrontDisconnected 被调用，其中的参数 nReason 描述了断线的原因：
* 4097（0x1001）：网络读失败
* 4098（0x1002）：网络写失败
* 8193（0x2001）：读心跳超时
* 8194（0x2002）：发送心跳超时
* 8195（0x2003）：收到不能识别的错误消息

客户端与服务端的连接断开有两种情况：
* 网络原因导致连接断开
* 服务端主动断开连接，有两种可能：
    - 客户端长时间没有从服务端接收报文，时间超时，默认的读超时为：120 秒，写超时为：60 秒，心跳超时为：80 秒。
    - 客户端建立的连接数超过限制

网络超时会导致服务端主动与客户端断开连接。默认的读超时为：120 秒，写超时为：60 秒，心跳超时为：80 秒。客户端与服务端连接断开后，交易接口会自动尝试重新连接，频率是每 5 秒一次。

**心跳机制**：综合交易平台采用心跳机制来确认客户端与服务端的连接是否正常。如果客户端没有从服务端接收报文，服务端会发送心跳。心跳机制目前仅在接口内部实现，并不会表现到客户端层面上来，即函数 OnHeartBeatWarning 并不会被调用。


## 数据阻塞

API 中回调函数被触发后必须快速返回，否则会导致其他数据的推送被阻塞，阻塞时间长了还有可能导致 API 发生崩溃，因此回调函数中不适合包含耗时较长的计算逻辑。例如某个 Tick 行情推送后，如果用户在回调函数中写了一些比较复杂的计算（循环计算等等），耗时超过 3 秒（vp.py 作者的一个经验），则在这个 3 秒中，其他的行情推送用户是收不到的（被阻塞了），且很可能 3 秒后会出现 API 崩溃（程序死掉）。这里的解决方案是使用生产者-消费者模型，在 API 中包含一个缓冲队列，当回调函数收到新的数据信息时只是简单存入缓冲队列中并立即返回，而数据信息的处理则由另一个工作线程来执行。

## 流量限制

CTP 仅对查询（ReqQry——）进行流量限制，对交易指令没有限制。交易接口中的查询操作的限制为：
* 每秒钟最多只能进行一次查询操作。
* 在途的查询操作最多只能有一个。（在途：查询操作从发送请求，到接收到响应为一个完整的过程。如果请求已经发送，但是未收到响应，则称该查询操作在途。）

返回值 -2 表示“未处理请求超过许可数”，-3 表示“每秒发送请求数超过许可数”。

报单流量限制是由期货公司通过在系统中配置相关参数实现限制的。不进行配置的情况下，默认流量限制为：
* 在一个连接会话（Session）中，每个客户端每秒钟最多只能发送 6 笔交易相关的指令（报单，撤单等）。
* 同一个账户同时最多只能建立 6 个会话（Session）。

报单操作超过限制时，不会有错误返回，只是报单会处于排队状态，等待处理。该限制包含包括报单，撤单，查询等所有请求操作。

## 数据加密

上线 CTP 的证券（期货）公司可以选择部署 ssl 前置供投资者客户端以 SSL 加密方式接入 CTP 交易系统。客户端选择接入 ssl 前置时使用如下方式注册 ssl 前置地址：`RegisterFront("ssl:// ssl 前置 IP 地址或域名:端口号");`

## 动态密码

动态密码（OneTimePassword）或动态令牌（时间令牌 TOTP）是 CTP 后台提供给证券（期货）公司投资者网上交易使用的一种更强的身份验证工具，该功能目前由投资者和证券（期货）公司选择使用。为 CTP 供应动态令牌的厂商有坚石诚信和飞天诚信。证券（期货）公司购买动态令牌后将随附的种子文件导入 CTP 后台，并将各令牌及种子分配给到指定的投资者，使用动态令牌的投资者在 ReqUserLogin 请求登录 CTP 时需要将动态令牌当时显示的字符填写到 OneTimePassword 字段，通过用户名/密码以及动态密码校验后方可登录成功。

## 名字服务器

上线 CTP 的证券（期货）公司可以选择部署名字服务器，客户端可以通过注册名字服务器（RegisterNameServer）自动选择 CTP 后台分配的前置机地址接入 CTP 交易系统，不再需要客户端直接注册 CTP 前置地址（RegisterFront）。

## 代理服务器

CTP 的 API 提供了对代理服务器的支持，包括 socks4、socks4a 及 socks5，客户端开发时只需通过传递给 API 不同的连接字符串就可实现，例如：RegisterFront("socks5:// 代理服务器 IP 地址或域名:端口号/user:pass@前置 IP 地址或域名:端口号");

## UDP 行情

创建行情 API 实例函数 `CreateFtdcMdApi(const char *pszFlowPath = "", const bool bIsUsingUdp=false)` 中， 参数 bIsUsingUdp 为行情模式，该参数缺省或置为 false 时使用 TCP 行情，否则为 UDP 行情。 无论使用 TCP 或是 UDP 行情，都必须注册相对应的 TCP 或 UDP 行情服务器地址。一般来说，CTP 的普通行情前置都为 TCP 行情服务器，使用 UDP 行情服务器需要向证券（期货）公司申请，且仅限专线或内网接入的投资者使用。

在注册行情前置服务器时，无论是 tcp 还是 udp，都必须使用 `RegisterFront("tcp://行情前置服务器 IP 地址或域名:端口号")` 的格式，因为 udp 传输存在不可靠性，所以在登录、订阅及接收第一次行情时仍然使用 tcp 方式；并且无需为 udp 方式提供配置节点参数，udp 仍然使用相同的地址和端口号。

## 资源释放

CThostFtdcMdApi 在使用 Release()退出清理对象时会出现死机，并且频率很高，可参考以下代码的释放顺序。

    template <class TUserApi>
    void CUserApiEnv<TUserApi>::UnInitialUserApi()
    {
        // 释放 Api 实例
        if (m_pUserApi)
        {
            m_pUserApi->RegisterSpi(NULL);
            m_pUserApi->Release();
            m_pUserApi = NULL;
        }
        // 释放 Spi 实例
        if (m_pUserSpiImpl)
        {
            delete m_pUserSpiImpl;
            m_pUserSpiImpl = NULL;
        }
    }

## 其他

集合竞价时不更新买卖价/量。集合竞价时发送的市价单会被交易所认为没有对手方，作为交易不成功来自动撤单。

“部成部撤”即“部分成交不在队列中”。 CTP 有一个自动挂起标志，如果设置了该标志，那么断线客户的未成交报单将被自动挂起，这时该报单的状态就是“未成交不在队列中”。自动挂起标志是从上期所系统沿用过来的东西，原来设计的“自动挂起”报单，可以撤单也可以通过“激活”指令让报单重新进入队列。目前请客户端将“自动挂起标志” 设置为 0，永远不挂起。

查询费率时返回品种只说明该合约的费率设置是取自对应品种的费率设置，后台没有对该合约进行特殊的设置。

如果发送一个报单委托价格在涨跌停板之外，超出涨跌停板的判断是在交易所处理，所以，CTP 收到报单就新增一条记录，然后收到交易所的 OnErrRtnOrder 后，修改委托表里的记录，触发 OnRtnOrder。OnErrRtnOrder 的作用是： CTP 在检查委托发现错误时，会给发出委托的投资者发出 OnRspInsertOrder，同时发出 OnErrRtnOrder 给相关的交易员，所以，作为投资者可以不关心 OnErrRtnOrder。

OnRtnOrder 有重复推送的问题。比如发一个单， OnRtnOrder 推了一个“已报”状态回来。然后我开始撤单，撤单一报入，OnRtnOrder 首先又推一个“已报”的状态回来，然后才是“撤消”的状态。投资者发出 1 个操作，都会收到 2 条回报。就是 CTP 应答一下，然后交易所又应答一下，都是把对应的委托状态从 OnRtnOrder 推回来。从 OnRtnOrder 中有没有办法区分这是从 CTP 返回的包还是从交易所返回的包，因为对客户端来说，所有返回包都由 CTP 发出。具体说：
* 发出 1 笔委托：
    1. CTP 发出“已提交”状态回报
    2. CTP 转发交易所的“未成交”状态回报
* 用户发出撤单；
    1. CTP 修改 Active User，再发出“未成交”状态回报；
    2. CTP 转发交易所的“已撤单”状态回报。

OnRtnInstrumentStatus 会通知当前交易所状态变化，据此区分报单状态回报或下单撤单反馈是不是由交易所小结休息引起的。

THOST_FTDC_OST_AllTraded、THOST_FTDC_OST_Canceled、THOST_FTDC_OST_NoTradeNotQueueing、THOST_FTDC_OST_PartTradedNotQueueing 是报单的最终状态，不会再改变了的。

目前只支持撤单，不支持报单的挂起、报单的激活、报单的修改这几个功能。

CTP 交易后台从管理平台或是交易所取得“合约生命周期状态”及“当前是否交易”两个字段的值，由于跟后台的设置关联，且 DCE 及 CZCE 并不会在盘中推送单个合约的状态，因此并不能通过这两个字段及时获得合约是否交易的信息。要查某个合约当天是否可以交易，可以查 IsTrading，要看现在是否可以交易，就看合约状态（仅上期所）或者交易所状态。

CTP 支持国内三家商品交易所正式对外开放的所有下单类型，大商所和郑商所提供套利交易指令和市价单，大商所提供止盈止损等条件单。CTP 在后台提供非交易时段的预埋单。

CTP 的历史报单只能通过期货公司管理人员在管理平台查询，客户端需要历史交易记录可以通过查询历史结算单的方式获取。

客户端认证是为了保证证券（期货）公司的投资者只能使用该公司认可的客户端产品接入 CTP 后台进行交易。投资者在使用第三方提供或是自行开发的客户端产品接入指定的证券（期货）公司 CTP 交易系统之前，必须向该公司提交用户端产品信息（UserProductInfo）并获得认证码（AuthCode）；之后在发起客户端认证请求时（ReqAuthenticate）填写正确的用户端产品信息及对应的认证码即可完成客户端认证。

CTP 最初设计时考虑了一套 CTP 系统供多家经纪公司使用的情形，CTP 则使用 BrokerID 从业务层面完全隔离不同经纪公司的交易、风控及结算用户的接入。 BrokerID 的具体取值请咨询开户的经纪公司。

CTP 中 UserID 和 InversterID 的区别：经纪公司交易员为投资者下单时，UserID 为操作员代码，InversterID 为投资者代码；投资者自己下单时，两者同为投资者代码。

# 附录

四家期货交易所的技术子公司各自开发了一套交易接口，各个交易所自己的技术子公司开发的系统是最好的选择，上期所产品用 CTP，中金所产品用飞马，大商所产品用飞创，郑商所产品用易盛。CTP 是唯一确认同时支持四家期货交易所期权做市商报价的系统。




## 类 CTP 交易 API

上期 CTP：
* 经纪商：期货公司、兴业证券
* 产品：期货、期货期权
* 特点：国内最早的针对程序化交易设计的 API 接口
* 申请：SimNow 网站申请

飞创：
* 经纪商：期货公司
* 产品：期货、期货期权
* 特点：交易大商所产品有速度优势
* 申请：通过大连飞创上的 QQ 群申请

飞马
* 经纪商：期货公司
* 产品：中金所的期货、期货期权
* 特点：交易中金所产品有速度优势
* 申请：通过飞马上的 QQ 群申请

飞鼠
* 经纪商：贵金属经纪公司
* 产品：期货、金交所的贵金属现货
* 特点：针对贵金属的期货和金交所现货套利交易有速度优势
* 申请：联系招金投资或恒邦冶炼申请

金仕达期权
* 经纪商：期货公司
* 产品：期货、期货期权、股票期权
* 特点：可以同时交易期货期权和股票期权，便于套利
* 申请：联系南华期货的客服申请

金仕达黄金
* 经纪商：贵金属经纪公司、银行
* 产品：金交所的贵金属现货
* 特点：部分银行可以提供的程序化接口，结合黄金 ETF 做套利有优势（非速度优势）
* 申请：在浦发银行或中国银行开户黄金 T+D 交易后找客户经理申请

XTP
* 经纪商：中泰证券
* 产品：股票、债券、ETF、证券期权
* 特点：高性能低延时证券柜台，适合日内 T+0 交易
* 申请：联系中泰证券申请

OANDA
* 经纪商：OANDA（美国外汇经纪商）
* 产品：外汇现货、CFD（全球股指、主要商品）
* 特点：开户交易的要求非常低（10 美金都行），适合新手低成本入门
* 申请：在 OANDA 官网开设模拟账户后即可使用，建议选择欧美国家（如英国），选择中国需要发送邮件申请开通 API 接入

IB
* 经纪商：Interactive Brokers（盈透证券）
* 产品：全球除了中国以外市场的几乎所有产品（股票、债券、期货、期权、外汇），部分中国股票可以通过沪股通形式参与
* 特点：一套接口交易全球市场
* 申请：在 IB 官网下载演示客户端后即可使用

# References

[<a id="cit-VNPY:Tutorial" href="#call-VNPY:Tutorial">VNPY:Tutorial</a>] 用 Python的交易员, ``_Python 量化交易平台开发教程系列_'',  .  [online](http://www.vnpy.org/pages/tutorial.html)

[<a id="cit-SimNow" href="#call-SimNow">SimNow</a>] 上海期货信息技术有限公司 , ``_CTP API 开发文档资料_'',  .  [online](http://simnow.com.cn/)

